In [8]:
import os
FILE_DIR = 'files'

In [12]:
def obabel(arg: str = "--help"):
    """Run obabel given a desired argument"""
    os.system(f"obabel {arg}")

def _add_tag_to_path(filepath, tag):
    base, ext = os.path.splitext(filepath)
    return f"{base}_{tag}{ext}"

In [21]:
def protonate_and_optimize_ligand(input_file: str, 
                                  ph: float|None = 7.2, 
                                  # save_dir: str = FILE_DIR,
                                  force_field: str = 'MMFF94',
                                  convergence_criteria: str = '0.00001',
                                  maximum_steps: int = 10000
                                 ):

    """Add hydrogens to molecules based on a given ph and perfrom energy minimization using obabel"""

    save_dir = FILE_DIR
    
    # PDB2PQR/PropKa for ptotein

    ph_str = str(ph).replace(".", "")             # 7.2 -> 72
    output_file_noH = _add_tag_to_path(input_file, 'noH')
    output_file_ph = _add_tag_to_path(input_file, f'ph{ph_str}')
    output_file_ph_min = _add_tag_to_path(input_file, f'ph{ph_str}_min') 
    log_dir = os.path.join(save_dir, 'obabel_optimize.log')

    if ph:
        # Remove hydrogens
        obabel(f"{input_file} -O {output_file_noH} -d")
        # Add hydrogens with the given ph
        obabel(f"{output_file_noH} -O {output_file_ph} -p {ph}")
        input_file = output_file_ph
    
    # Perform energy minimization
    obabel(f"""{input_file} -O {output_file_ph_min} \
    --best --minimize --ff {force_field} --steps {maximum_steps} --sd \
    --crit {convergence_criteria} > {log_dir} 2>&1""")

In [22]:
protonate_and_optimize_ligand("files/ibuprofen_S.sdf", ph=7.4)

1 molecule converted
1 molecule converted
